In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt


In [ ]:
url = "https://raw.githubusercontent.com/funnyPhani/HouseData/main/kc_house_data.csv"
data = pd.read_csv(url)
data.drop(['id', 'date'], axis=1, inplace=True)
assert data.isnull().sum().sum() == 0, "Data contains missing values!"

In [ ]:
X = data.drop('price', axis=1)
y = np.log1p(data['price'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#(Standardization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

def build_model(hp):
    model = Sequential()

    model.add(Dense(units=hp.Int('units_input', 64, 128, step=32),
                    activation='relu',
                    kernel_regularizer=l2(0.001),
                    input_shape=(X_train.shape[1],)))
    model.add(BatchNormalization())

    for i in range(hp.Int('num_layers', 1, 2)):  # 1–2 layers max
        model.add(Dense(units=hp.Int(f'units_{i}', 64, 128, step=32),
                        activation='relu',
                        kernel_regularizer=l2(0.001)))
        model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.3, step=0.1)))

    model.add(Dense(1, activation='linear'))

    model.compile(
        optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
        loss='mse',
        metrics=['mae']
    )

    return model


In [ ]:
#Hyperparameter tuning using Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='house_price_mlp'
)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
tuner.search(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stop], verbose=1)

Trial 10 Complete [00h 07m 09s]
val_loss: 33552435200.0

Best val_loss So Far: 13615249408.0
Total elapsed time: 01h 11m 00s


In [ ]:
#Get the best model
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:")
print(f"Units input: {best_hp.get('units_input')}")
print(f"Number of layers: {best_hp.get('num_layers')}")
for i in range(best_hp.get('num_layers')):
    print(f"Layer {i} units: {best_hp.get(f'units_{i}')}")
    print(f"Layer {i} dropout: {best_hp.get(f'dropout_{i}')}")
print(f"Learning rate: {best_hp.get('learning_rate')}")

best_model = tuner.hypermodel.build(best_hp)

Best hyperparameters:
Units input: 160
Number of layers: 3
Layer 0 units: 160
Layer 0 dropout: 0.2
Layer 1 units: 96
Layer 1 dropout: 0.30000000000000004
Layer 2 units: 256
Layer 2 dropout: 0.2
Learning rate: 0.002130209316625156


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Train the best model on full training data
history = best_model.fit(X_train, y_train, epochs=100, validation_split=0.2,
                         callbacks=[early_stop], verbose=1)

#Evaluate model on test set
y_pred = best_model.predict(X_test).flatten()

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.2f}")
print(f"Test MAE: {mae:.2f}")
print(f"Test R^2 Score: {r2:.4f}")

Epoch 1/100
433/433 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 210626723840.0000 - mae: 321944.1250 - val_loss: 31080888320.0000 - val_mae: 115863.5938
Epoch 2/100
433/433 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 34154233856.0000 - mae: 118853.9219 - val_loss: 28635805696.0000 - val_mae: 110389.3750
Epoch 3/100
433/433 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 32827043840.0000 - mae: 115629.5312 - val_loss: 27989065728.0000 - val_mae: 108844.3203
Epoch 4/100
433/433 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 33805881344.0000 - mae: 117005.1641 - val_loss: 27408590848.0000 - val_mae: 106592.7500
Epoch 5/100
433/433 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 34335545344.0000 - mae: 116061.5859 - val_loss: 26608257024.0000 - val_mae: 105503.3594
Epoch 6/100
433/433 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 32901781504.0000 - mae: 115023.6328 - val_loss: 29294067712.0000 - val_mae: 111674.3516
Epoch 7/100
433/433 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 35109462016.0000 - mae: 116172.0234 - val_lo